# Form the NB Regression Random Intercept Model: Simulated set

Next we may try joint estimation using Newton's + IPOPT. 

    I create new structures `NBCopulaVCObs`, `NBCopulaVCModel`
    



In [26]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [2.0, 1.0, 2.0, 6.0, 1.0]
 [8.0, 5.0, 3.0, 5.0, 2.0]
 [2.0, 4.0, 4.0, 10.0, 2.0]
 [4.0, 3.0, 7.0, 4.0, 8.0]
 [10.0, 10.0, 9.0, 1.0, 1.0]
 [4.0, 4.0, 3.0, 4.0, 3.0]
 [9.0, 6.0, 5.0, 4.0, 4.0]
 [6.0, 1.0, 3.0, 6.0, 0.0]
 [4.0, 7.0, 3.0, 3.0, 3.0]
 [5.0, 5.0, 3.0, 6.0, 2.0]
 [7.0, 6.0, 5.0, 2.0, 3.0]
 [3.0, 2.0, 7.0, 1.0, 4.0]
 [2.0, 3.0, 2.0, 2.0, 6.0]
 ⋮
 [2.0, 3.0, 1.0, 4.0, 2.0]
 [1.0, 3.0, 6.0, 2.0, 2.0]
 [2.0, 1.0, 3.0, 5.0, 1.0]
 [2.0, 3.0, 1.0, 1.0, 5.0]
 [2.0, 3.0, 4.0, 4.0, 10.0]
 [2.0, 3.0, 6.0, 2.0, 6.0]
 [2.0, 3.0, 1.0, 8.0, 2.0]
 [8.0, 0.0, 6.0, 3.0, 5.0]
 [5.0, 3.0, 2.0, 3.0, 6.0]
 [5.0, 6.0, 6.0, 9.0, 5.0]
 [0.0, 2.0, 1.0, 4.0, 3.0]
 [3.0, 6.0, 1.0, 3.0, 3.0]

In [27]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);

In [28]:
initialize_model!(gcm)
@show gcm.β
@show gcm.Σ

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -130470.51823172439 39999
2 -130470.51823172439 -130470.51823172439 39999
initializing variance components using MM-Algorithm
gcm.Σ = [4.787417886202995e-7, 2.3125551854877464e-7]
gcm.β = [1.505897960765723]
gcm.Σ = [4.787417886202995e-7, 2.3125551854877464e-7]


2-element Vector{Float64}:
 4.787417886202995e-7
 2.3125551854877464e-7

In [29]:
β_true

1.4552872326068422

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [30]:
GLMCopula.loglikelihood!(gcm, true, true)

-130470.52975987685

## Try Quasi-Newton after specifying gradient of r

to-do: 

1. added field $r$ to model structure $gcm.r$, change access to r this way instead of through the distribution
2. add $gc.\nabla r$ in loglikelihood function


## Newton after specifying the hessian term too

3. add $gc.Hr$ in loglikelihood function

In [31]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

-119245.32538920302

In [33]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.505897968418259; true β = 1.4552872326068422
estimated variance component 1 = 0.0; true variance component 1 = 0.1
estimated variance component 2 = 0.0; true variance component 2 = 0.5
estimated r = 5.6002812821901005; true r = 10


In [34]:
# Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

-119245.32538920305

In [35]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 1.5058979684001337; true β = 1.4552872326068422
estimated variance component 1 = 0.0; true variance component 1 = 0.1
estimated variance component 2 = 0.0; true variance component 2 = 0.5
estimated r = 5.600281282475263; true r = 10
